In [1]:
import time
from threading import Thread
from functools import reduce
import re
import pickle
from pathlib import Path
import sqlite3
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

In [2]:
class BackupApiResp:
    def __init__(self,path_file_bk, max_days_to_valid_data = 720): 
        self.__con = sqlite3.connect(path_file_bk)
        self.__con.row_factory = sqlite3.Row
        self.days_to_expires = max_days_to_valid_data 
        self.today = datetime.now().strftime("%Y-%m-%d %H:%M")
        self.__init_db()
    
    def __init_db(self):
        sql_table_def = (
            'CREATE TABLE IF NOT EXISTS ' 
            'anuncios_resumo ('
                'url_lista TEXT,'
                'categoria_completa TEXT,'
                'categoria_atual TEXT,'
                'localizacao_completa TEXT,'
                'data_coleta_dados DATETIME,'
                'titulo_anuncio TEXT,'
                'preco_anuncio FLOAT,'
                'anuncio_profissional INTEGER,'
                'url_anuncio TEXT,'
                'data_publicacao_anuncio DATETIME'
            ');'
        )

        sql_index1_def = (
            'CREATE INDEX IF NOT EXISTS '
            'index_url_anuncio_ar ON '
            'anuncios_resumo ('
                'url_anuncio ASC'
            ');'
        )

        sql_index2_def = (
            'CREATE INDEX IF NOT EXISTS '
            'index_url_lista_ar ON '
            'anuncios_resumo ('
                'url_lista ASC'
            ');'
        )

        self.__con.execute(sql_table_def)
        self.__con.execute(sql_index1_def)
        self.__con.execute(sql_index2_def)
        self.__con.commit()

    def add_row_anuncios_resumo(self,url_lista,categoria_completa,categoria_atual,localizacao_completa,titulo_anuncio,preco_anuncio, anuncio_profissional,url_anuncio,data_publicacao_anuncio):
        sql_insert_data = (
            "INSERT INTO anuncios_resumo (url_lista,categoria_completa,categoria_atual,localizacao_completa,data_coleta_dados,titulo_anuncio,preco_anuncio, anuncio_profissional,url_anuncio,data_publicacao_anuncio) "
            "VALUES ("
                f"'{url_lista}',"
                f"'{categoria_completa}',"
                f"'{categoria_atual}',"
                f"'{localizacao_completa}',"
                f"'{self.today}',"
                f"'{titulo_anuncio}',"
                f"{preco_anuncio},"
                f"{anuncio_profissional},"
                f"'{url_anuncio}',"
                f"'{data_publicacao_anuncio}' "
            ");"
        )
        self.__con.execute(sql_insert_data)
        self.__con.commit()
    
    def date_is_valid(self,date:str):
        data_age_days = (datetime.now()-datetime.fromisoformat(date)).days
        return (data_age_days <= self.days_to_expires)

    def has_url_ad_in_anuncio_resumo(self,url_ad):
        sql = f""" select url_anuncio from anuncios_resumo
                    where url_anuncio = '{url_ad}'
                    limit 1 """

        result = self.__con.execute(sql).fetchone()
        return True if result else False







In [3]:
def print_verbose_function(verbose):
    if verbose:
        return print
    else:
        return lambda x: None

In [4]:
def convert_date_olx_to_datetime_str(texto_data):
    MONTHS = {'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4,  'mai': 5,  'jun': 6,
          'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12}
    data, hora = texto_data.lower().split(',')
    data_padrao = ""
    
    if 'ontem' == data :
        data_padrao = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    elif 'hoje' == data :
        data_padrao = datetime.now().strftime('%Y-%m-%d')
    else:    
        dataP = data.split(' ')
        dia_mes = int(dataP[0][0:2])
        mes = MONTHS[dataP[1]]
        ano_atual = int(datetime.now().strftime('%Y'))
        mes_atual = int(datetime.now().strftime('%m'))
        ano = ano_atual if mes_atual >= mes else ano_atual - 1
        data_padrao = datetime(year=ano, month=mes, day=dia_mes).strftime('%Y-%m-%d')
    
    return data_padrao + hora

In [5]:
# #Teste
# backup = BackupApiResp("./banco_backup.db")
# backup.add_row_anuncios_resumo('www.teste','hobbys;livros','livros','Brasil;Minas;31','mágico de oz',200.98, 1,'www.teste2','2022-08-30 15:21')
# backup.has_url_ad_in_anuncio_resumo('www.teste2')

In [6]:
headers_olx =  { 
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                "accept-encoding": "gzip, deflate, br",
                "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
                "origin": "https://olx.com.br/",
                "referer": "https://olx.com.br/",
                "sec-ch-ua": "\" Not A;Brand\";v=\"99\", \"Chromium\";v=\"96\", \"Google Chrome\";v=\"96\"",
                "sec-ch-ua-mobile": "?0",
                "sec-ch-ua-platform": "\"Windows\"",
                "sec-fetch-dest": "empty",
                "sec-fetch-mode": "cors",
                "sec-fetch-site": "same-site",
                "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
                }

In [7]:
def get_retroactive_links_in_div(div_class,url,verbose=True,only_endpoints=False):
    """
    this code is used to get the retroactive links in the div class
    """
    session = requests.Session()
    all_url_visited = []
    end_links = []
    error_links = []
    iprint = print_verbose_function(verbose)

    def get_links_in_current_div(url_base,suffix='',qt_try=1):
        nonlocal session
        if url_base not in all_url_visited:
            all_url_visited.append(url_base)   
        iprint("*****url_base: "+ url_base)
        
        try:
            resp = session.get(url_base+suffix, headers=headers_olx, timeout=15)
            bsObj = BeautifulSoup(resp.text, "html.parser")
            items = bsObj.find("div", {"class":div_class})
            qt_reg_adds = 0
            if items:
                links_in_div = [*map(lambda x: x['href'], items.find_all("a", href=True))]
                
                for link in links_in_div:
                    if link not in all_url_visited:
                        iprint('+Redirecionamento:'+url_base+'->' + link)
                        get_links_in_current_div(link,'/')
                        qt_reg_adds+=1
                
            else:
                print('warning-no-class:' + url_base)
            
            if qt_reg_adds == 0:
                if url_base not in end_links:
                    iprint('----end_link:'+url_base) 
                    end_links.append(url_base)
        
        except Exception as e:
            print('!!! Warning, qt attempts:'+str(qt_try)+' in:'+url_base)
            print(str(e))
            print('!!! Restart session and Wait 10 seconds\n\n')
            time.sleep(10)
            session = requests.Session()
            if qt_try < 6:
                get_links_in_current_div(url_base,suffix,qt_try+1)
            else:
                print('!!! Error, qt attempts:'+str(qt_try)+' in:'+url_base)
                error_links.append(url_base)

    get_links_in_current_div(url)       
    
    return all_url_visited if only_endpoints == False else end_links

In [8]:
# _test_endlink = get_retroactive_links_in_div('sc-1ncgzjx-0','https://mg.olx.com.br/belo-horizonte-e-regiao/imoveis/venda',True,True)

In [9]:
def get_unique_category_urls():
    """
    Return only the most specific category links 
    *(general categories are not returned and yours advertisements are included in specific categories)
    * is necessary get specific categories because the general categories are limited in 5k advertisements per location
    """
    url_base = "https://www.olx.com.br/brasil"
    class_category_menu = "jx24x3-2"

    all_url_visited = get_retroactive_links_in_div(class_category_menu,url_base) # end_links = False -> because there are siblings links in categories menu

    most_specific_category = []

    # Is necessary threat all links using Regex to get the most specific category
    for link_ref in all_url_visited:
        add = True
        for link_comp in all_url_visited:
            if re.match(link_ref, link_comp) and link_ref != link_comp:
                add = False
                break
        if add:
            most_specific_category.append(link_ref)

    most_specific_category.remove(url_base)       
    
    return  most_specific_category   

In [10]:
def get_start_urls_by_location_threads(urls_base,qt_threads=10,verbose=False):
    """
    Return a list of urls to be scraped (inside a specific region)
    Is necessary to use threads because the OLX website has a lot of categories and the scraping is slow
    """
    def _get_retroactive_links_in_div(div_class,url,results,index):
        #is necessay to use this function because the get_retroactive_links_in_div function is not thread safe
        results[index] = get_retroactive_links_in_div(div_class,url,verbose,True)
    
    class_location_menu = "sc-1ncgzjx-0"
    start_urls = []
    threads = [None] * qt_threads
    results = [None] * qt_threads
    qt_urls_base = len(urls_base)
    print('Start get_start_urls_scraping_threads - This Function take many time to finish')
    for index in range(0,qt_urls_base,qt_threads):
        for i in range(qt_threads):
            if (index+i) < qt_urls_base:
                threads[i] = Thread(target=_get_retroactive_links_in_div, args=(class_location_menu,urls_base[index+i],results,i))
                threads[i].start()
        for i in range(qt_threads):
            if (index+i) < qt_urls_base:
                threads[i].join()
                print(f"Category {urls_base[index+i]} ended: {index+i+1} / {qt_urls_base}")
                start_urls.extend(results[i])
        # start_urls = sum(results,start_urls)
        threads = [None] * qt_threads
        results = [None] * qt_threads

    return [*filter(lambda x: x != None, start_urls)]

In [11]:
# Test = get_start_urls_scraping_threads(['https://ac.olx.com.br/hobbies-e-colecoes'])

In [12]:
def get_start_urls_by_location(urls_base,verbose=False):
    """
    Return a list of urls to be scraped (inside a specific region)
    """
    class_location_menu = "sc-1ncgzjx-0"
    all_url_visited = []
    for url_base in urls_base:
      qt_url_base = len(urls_base)
      percetil_end = len(all_url_visited)/qt_url_base*100
      print(f'Percentual Concluido:{percetil_end:.2f}% - {len(all_url_visited)} de {qt_url_base}')
      all_url_visited.extend(get_retroactive_links_in_div(class_location_menu,url_base,verbose,True))
    return all_url_visited

In [13]:
unique_category_urls = []
if not Path('./unique_category_urls.pkl').is_file():
    unique_category_urls = get_unique_category_urls()
    with open('./unique_category_urls.pkl', 'wb') as file:
        pickle.dump(unique_category_urls, file)
else :
    with open('./unique_category_urls.pkl', 'rb') as file:
        unique_category_urls = pickle.load(file)   

In [14]:
start_urls_scraping = []
if not Path('./start_urls_scraping.pkl').is_file():
    start_urls_scraping = get_start_urls_by_location_threads(unique_category_urls,12)
    with open('./start_urls_scraping.pkl', 'wb') as file:
        pickle.dump(start_urls_scraping, file) 
else :
    with open('./start_urls_scraping.pkl', 'rb') as file:
        start_urls_scraping = pickle.load(file)